# Sclblpy

#### Sclblpy is the core python package provided by Scailable to interact with our API. It allows you to create a scailable account,  add a device to your organisation, upload a model to sclbl admin and assign it to an existing device and many more other functionnalities.

In [1]:
import sclblpy as sp

### Create an account 

In [3]:
print(sp.register_('your name', 'your company', 'your_email@gmail.com', 'stroong_password'))
# It is recommended to also add your job title and phone number when registering

True


### Already have an account ? log in

In [15]:
print(sp.log_in('your_email@gmail.com', 'stroong_password'))

True


### Register a device

In [17]:
print(sp.add_device('Device_name'))
# You can also provide the runtime, and the device type and serial

{'UUID': '5dbd022b-7eb7-4ebb-a3d6-b7f7f22e68da', 'Name': 'Device_name', 'Serial': None, 'Runtime': None, 'Type': None, 'InsertedAt': '2023-05-23T20:16:14.965041Z', 'UpdatedAt': '2023-05-23T20:16:14.965041Z', 'OnlineStatus': {'Online': False, 'OnlineAt': None}, 'FunctionCount': 0, 'Licenses': [{'UUID': '0b30a2d2-d46f-4f29-a1f8-6eab6a9e4fda', 'Agreement': {'Name': 'AI Manager License', 'Kind': 'trial'}, 'LicenseKey': '3ee4b776-368e-43ba-a480-d34837740fcb', 'StartsAt': '2023-05-23T20:16:15.003716Z', 'ExpiresAt': '2023-08-21T20:16:15.003716Z'}], 'Groups': None, 'Functions': []}
Your device was successfully added to Scailable!
You can use the '_all_devices()' function to list all your devices. 

True


#### You can use the get_all_devices function to list all devices of your organisation

In [18]:
devices = sp.get_all_devices()
print(devices)

[{'UUID': '5dbd022b-7eb7-4ebb-a3d6-b7f7f22e68da', 'Name': 'Device_name', 'Serial': None, 'Runtime': None, 'Type': None, 'InsertedAt': '2023-05-23T20:16:14.965041Z', 'UpdatedAt': '2023-05-23T20:16:14.965041Z', 'OnlineStatus': {'Online': False, 'OnlineAt': None}, 'FunctionCount': 0, 'Licenses': None, 'Groups': None}]


#### In order to use a device, you'll need its uuid

In [20]:
for device in devices:
    if device['Name'] == "Device_name":
        device_uuid = device['UUID']
        print(device_uuid)

5dbd022b-7eb7-4ebb-a3d6-b7f7f22e68da


### (Optional) Build a model

##### While most .onnx files will work out of the box with sclblpy, many of you will aim to use sclblpy to deploy some kind of Neural Network trained in e.g. Pytorch. We'll show you briefly how to export a Neural Network from the torch.nn.Module ... module to onnx, but feel free to use your own model, as long as it can be exported to onnx (one caveat: Not every operator is currently supported, find a list of the ones that work [here](https://github.com/scailable/sclblonnx/blob/master/sclblonnx/supported_onnx.json).

##### Our example won't even be trained, since we're only interested in the deployment part, essentially we'll be doing the first few steps from https://pytorch.org/tutorials/advanced/super_resolution_with_onnxruntime.html; feel free to do the rest of the tutorial as well!

##### NOTE: pixel_shuffle() is not yet supported by our runtime, so we have to remove it before conversion

In [6]:
import torch
import torch.nn as nn
import torch.nn.init as init


class SuperResolutionNet(nn.Module):
    def __init__(self, upscale_factor, inplace=False):
        super(SuperResolutionNet, self).__init__()

        self.relu = nn.ReLU(inplace=inplace)
        self.conv1 = nn.Conv2d(1, 64, (5, 5), (1, 1), (2, 2))
        self.conv2 = nn.Conv2d(64, 64, (3, 3), (1, 1), (1, 1))
        self.conv3 = nn.Conv2d(64, 32, (3, 3), (1, 1), (1, 1))
        self.conv4 = nn.Conv2d(32, upscale_factor ** 2, (3, 3), (1, 1), (1, 1))
        #self.pixel_shuffle = nn.PixelShuffle(upscale_factor)

        self._initialize_weights()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.relu(self.conv3(x))
        #x = self.pixel_shuffle(self.conv4(x))
        return x

    def _initialize_weights(self):
        init.orthogonal_(self.conv1.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv2.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv3.weight, init.calculate_gain('relu'))
        init.orthogonal_(self.conv4.weight)

# Create the super-resolution model by using the above model definition.
torch_model = SuperResolutionNet(upscale_factor=3)

print(torch_model)

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)


##### So now we have our model, we'll call torch.eval() before we convert it to onnx

In [7]:
batch_size = 1    # just a random number


# set the model to inference mode
torch_model.eval()

SuperResolutionNet(
  (relu): ReLU()
  (conv1): Conv2d(1, 64, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv4): Conv2d(32, 9, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
)

In [22]:
# Input to the model
x = torch.randn(batch_size, 1, 224, 224, requires_grad=True)
torch_out = torch_model(x)

# Export the model
torch.onnx.export(torch_model,               # model being run
                  x,                         # model input (or a tuple for multiple inputs)
                  "super_resolution.onnx",   # where to save the model (can be a file or file-like object)
                  export_params=True,        # store the trained parameter weights inside the model file
                  opset_version=10,          # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'], # the model's output names
                  dynamic_axes={'input' : {0 : 'batch_size'},    # variable length axes
                                'output' : {0 : 'batch_size'}})

============== Diagnostic Run torch.onnx.export version 2.0.1+cpu ==============
verbose: False, log level: Level.ERROR
======================= 0 NONE 0 NOTE 0 WARNING 0 ERROR ========================



###  Upload an onnx model

In [23]:
print(sp.upload_model('super_resolution model', 'some doc', 'input driver', 'output driver', path='super_resolution.onnx'))

Your ONNX file was successfully uploaded to Scailable!
NOTE: After transpiling, we will send you an email and your model will be available at https://admin.sclbl.net.
Or, alternatively, you can use the '_functions_list()' function to list all your uploaded models. 

True


### (optional) wait for a few seconds to give the server time to convert the model

In [24]:
import time

time.sleep(15)


#remove file
import os
if os.path.exists("super_resolution.onnx"):
    os.remove("super_resolution.onnx")
else:
    print("The file does not exist")

##### To see your uploaded model you can use the below function to list all models of your organisation

In [25]:
models = sp.get_all_models()
print(models)

[{'Organisation': {'UUID': 'de7521e4-7e7e-47df-82d7-4aa998c65f52', 'Name': 'your company'}, 'Catalogues': [], 'UUID': '14f08006-d189-4de6-b5d4-126111962bb2', 'Name': 'super_resolution model', 'Status': 'ok', 'Alias': '', 'Documentation': 'some doc', 'InputDriver': 'input driver', 'InputDriverDetails': {}, 'OutputDriver': 'output driver', 'OutputDriverDetails': {}, 'NamedInput': [{'DataType': 'float', 'Name': 'input', 'Shape': [1, 1, 224, 224]}], 'NamedOutput': [{'DataType': 'float', 'Name': 'output', 'Shape': [1, 32, 224, 224]}], 'DeviceCount': 0, 'UpdatedAt': '2023-05-23T20:17:21.303515Z', 'Code': {'Size': 429632}, 'SyncInformation': {}, 'Source': {'Name': None, 'Url': None}}]


In [27]:
# Get the uuid of your model
for model in models:
    if model['Name'] == 'super_resolution model':
        print(model['UUID'])

14f08006-d189-4de6-b5d4-126111962bb2


### Update a model

In [ ]:
print(sp.update_model('uuid', 'new_name', 'new doc', 'input_driver', 'output_driver'))

### Assign a model to a device

In [28]:
print(sp.assign_model_to_device('5dbd022b-7eb7-4ebb-a3d6-b7f7f22e68da', '14f08006-d189-4de6-b5d4-126111962bb2'))

Your model was successfully assigned to the device 5dbd022b-7eb7-4ebb-a3d6-b7f7f22e68da
True


### Delete a model

In [ ]:
print(sp.delete_model(('14f08006-d189-4de6-b5d4-126111962bb2')))

## Other functionalities

### Get all groups of your organisation (a group is a set of models)

In [ ]:
print(sp.get_groups())

### Add a device to a group

In [ ]:
print(sp.add_devices_to_group('uuid of the group', ['devices']))